In [ ]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import pickle
import time
import hnswlib


model_name = 'quora-distilbert-multilingual'
model = SentenceTransformer(model_name)

dataset_path="/home/ubuntu/youtube_video_list.tsv"
max_corpus_size = 93

embedding_cache_path = 'youtube-embeddings-{}-size-{}.pkl'.format(model_name.replace('/', '_'), max_corpus_size)


embedding_size = 100    #Size of embeddings
top_k_hits = 3         #Output k hits

#Check if embedding cache path exists
if not os.path.exists(embedding_cache_path):
    # Check if the dataset exists. If not, download and extract
    # Download dataset if needed
    if not os.path.exists(dataset_path):
        print("Download dataset")
        util.http_get(url, dataset_path)

    # Get all unique sentences from the file
    corpus_sentences = set()
    
    with open(dataset_path, encoding='utf8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            #print(row)
            corpus_sentences.add(row['Youtube_title'])
            if len(corpus_sentences) >= max_corpus_size:
                break

    corpus_sentences = list(corpus_sentences)
    print("Encode the corpus. This might take a while")
    corpus_embeddings = model.encode(corpus_sentences, show_progress_bar=True, convert_to_numpy=True)

    print("Store file on disc")
    with open(embedding_cache_path, "wb") as fOut:
        pickle.dump({'sentences': corpus_sentences, 'embeddings': corpus_embeddings}, fOut)
else:
    print("Load pre-computed embeddings from disc")
    with open(embedding_cache_path, "rb") as fIn:
        cache_data = pickle.load(fIn)
        corpus_sentences = cache_data['sentences']
        corpus_embeddings = cache_data['embeddings']

#Defining our hnswlib index
index_path = "./hnswlib.index"
#We use Inner Product (dot-product) as Index. We will normalize our vectors to unit length, then is Inner Product equal to cosine similarity
index = hnswlib.Index(space = 'cosine', dim = embedding_size)

if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
else:
    ### Create the HNSWLIB index
    print("Start creating HNSWLIB index")
    index.init_index(max_elements = len(corpus_embeddings), ef_construction = 400, M = 64)

    # Then we train the index to find a suitable clustering
    index.add_items(corpus_embeddings, list(range(len(corpus_embeddings))))

    print("Saving index to:", index_path)
    index.save_index(index_path)

# Controlling the recall by setting ef:
index.set_ef(50)  # ef should always be > top_k_hits

######### Search in the index ###########

print("Corpus loaded with {} sentences / embeddings".format(len(corpus_sentences)))

while True:
    inp_question = input("Please enter a question: ")

    start_time = time.time()
    question_embedding = model.encode(inp_question)

    #We use hnswlib knn_query method to find the top_k_hits
    corpus_ids, distances = index.knn_query(question_embedding, k=top_k_hits)

    # We extract corpus ids and scores for the first query
    hits = [{'corpus_id': id, 'score': 1-score} for id, score in zip(corpus_ids[0], distances[0])]
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    end_time = time.time()

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))

    # Approximate Nearest Neighbor (ANN) is not exact, it might miss entries with high cosine similarity
    # Here, we compute the recall of ANN compared to the exact results
    correct_hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k_hits)[0]
    correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
    
    print("correct hits", correct_hits)
    print("correct_hits_ids", correct_hits_ids)

    knn_corpus_ids = set([hit['corpus_id'] for hit in hits])
        
    if len(knn_corpus_ids) != len(correct_hits_ids):
        print("Approximate Nearest Neighbor returned a different number of results than expected")

    recall = len(knn_corpus_ids.intersection(correct_hits_ids)) / len(correct_hits_ids)
    
    print("Results:")
    for hit in correct_hits[0:top_k_hits]:
        if hit['corpus_id'] not in knn_corpus_ids:
            print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))
    print("\n\n========\n")

Encode the corpus. This might take a while


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Store file on disc
Loading index...
Corpus loaded with 93 sentences / embeddings
Please enter a question: how is Biden doing in his presidency
Input question: how is Biden doing in his presidency
Results (after 0.518 seconds):
correct hits [{'corpus_id': 5, 'score': 0.8723465204238892}, {'corpus_id': 69, 'score': 0.8607958555221558}, {'corpus_id': 78, 'score': 0.8515018224716187}]
correct_hits_ids {69, 5, 78}
Results:
	0.872	Biden adviser defends the president’s economy|https://www.youtube.com/watch?v=mKYi3TvX1U8
	0.861	What's behind the Biden family's 'opulent' lifestyle?|https://www.youtube.com/watch?v=3OKIvDDNAC8
	0.852	Perino: You have to wonder who the White House spin on 'recession' is for|https://www.youtube.com/watch?v=FrPEzSRhmEs



Please enter a question: Who are all in Senate
Input question: Who are all in Senate
Results (after 0.344 seconds):
correct hits [{'corpus_id': 87, 'score': 0.8043516278266907}, {'corpus_id': 74, 'score': 0.780350923538208}, {'corpus_id': 79, 'scor